# INSTALLAZIONE LIBRERIE

In [1]:
!pip install spacy
!python -m spacy download en_core_web_sm
!python -m spacy download it_core_news_sm
!pip install translate-toolki
!pip install keras
!pip install tensorflow
!pip install translate-toolkit
!pip install sacrebleu
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 69.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 44.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
ERROR: Could not find a version that satisfies the requirement translate-toolki (from versions: none)
ERROR: No matching distribution found for translate-toolki
   ━

In [2]:
from google.colab import drive
from translate.storage import tmx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
import tensorflow as tf
import keras
from keras.preprocessing.sequence import pad_sequences
import sklearn
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from transformers import MarianMTModel, MarianTokenizer
import sacrebleu
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score

# DATASET

In [3]:
drive.mount('/content/drive')
file_path = "/content/drive/MyDrive/en-it.tmx"
sentence_pairs = []

Mounted at /content/drive


In [4]:
# Open the file in read mode
with open(file_path, 'rb') as f:
  tmx_file = tmx.tmxfile(f)

# Iterate through the translation units and store sentence pairs
for unit in tmx_file.units:
    source_text = unit.source
    target_text = unit.target
    sentence_pairs.append((source_text, target_text))

# Create a DataFrame from the sentence pairs
df = pd.DataFrame(sentence_pairs, columns=['Source', 'Target']);

# PREPROCESSING

In [5]:
# Funzione di pulizia per rimuovere caratteri speciali e gestire gli accenti
def clean_text(text):
    text = re.sub(r"http\S+|www.\S+", "", text)
    text = re.sub(r"[^a-zA-ZàèéìòùÀÈÉÌÒÙçÇ]", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

# Applichiamo la pulizia su Source e Target
df['Source_clean'] = df['Source'].apply(clean_text)
df['Target_clean'] = df['Target'].apply(clean_text)

# Rimuove le righe con frasi vuote
df = df[df['Source_clean'].str.strip() != '']
df = df[df['Target_clean'].str.strip() != '']

# Rimuove frasi troppo corte (esempio: meno di 3 parole)
df = df[df['Source_clean'].apply(lambda x: len(x.split()) >= 3)]
df = df[df['Target_clean'].apply(lambda x: len(x.split()) >= 3)]

# Verifica la presenza di eventuali valori nulli
print(df.isnull().sum())

# Rimozione duplicati
df.drop_duplicates(subset=['Source_clean', 'Target_clean'], inplace=True)

# Verifica dei risultati puliti
df[['Source_clean', 'Target_clean']].head(10)

Source          0
Target          0
Source_clean    0
Target_clean    0
dtype: int64


,Source_clean,Target_clean
1,There s a tight and surprising link between th...,Esiste uno stretto e sorprendente legame tra l...
2,fish health mission blue oceans science,fish health mission blue oceans science
4,Stephen Palumbi Following the mercury trail,Stephen Palumbi Sulle tracce del mercurio
5,It can be a very complicated thing the ocean,Può essere una cosa davvero complicata l oceano
6,And it can be a very complicated thing what hu...,E può essere una cosa davvero complicata la sa...
7,And bringing those two together might seem a v...,E mettere insieme queste due cose può sembrare...
8,And those simple themes aren t really themes a...,E questi semplici aspetti non riguardano le co...
9,And I m going to start with this one If momma ...,E vorrei partire da questa mamma infelice tutt...
10,We know that right We ve experienced that,Lo sappiamo tutti giusto Ci siamo passati tutti
11,And if we just take that and we build from the...,E se prendiamo questa idea e partiamo da qui p...


In [6]:
# Rimuove frasi troppo lunghe (più di 96 parole)
df = df[df['Source'].apply(lambda x: len(x.split()) <= 96)]
df = df[df['Target'].apply(lambda x: len(x.split()) <= 96)]
# Stampa la lunghezza del dataset
print(f"Dataset length after the removal of too long sentences: {len(df)}")

Dataset length after the removal of too long sentences: 153535


# CAMPIONAMENTO DATASET

In [7]:
# Campionamento casuale del 5% del dataset
sample_size = int(0.05 * len(df))  # Modifica a 0.10 per il 10%
sampled_data = df.sample(n=sample_size, random_state=42).reset_index(drop=True)

# MODELLO LLM

In [9]:
#from transformers import T5Tokenizer, T5ForConditionalGeneration

#model_name = 't5-small'
#tokenizer = T5Tokenizer.from_pretrained(model_name)
#model = T5ForConditionalGeneration.from_pretrained(model_name)


In [10]:
# Inizializza modello e tokenizer per la traduzione EN -> IT
model_name = 'Helsinki-NLP/opus-mt-en-it'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/789k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/814k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/343M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

# TRADUZIONE FRASI

In [13]:
# Funzione per tradurre in batch
def batch_translate(texts, batch_size=20):
    results = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True)
        translated = model.generate(**inputs)
        results.extend([tokenizer.decode(t, skip_special_tokens=True) for t in translated])
    return results

# Applica la traduzione al sottoinsieme campionato del dataset
sampled_data['translated'] = batch_translate(sampled_data['Source_clean'].tolist(), batch_size=20)

# VALUTAZIONE

In [33]:
# Funzione di normalizzazione del testo
#def normalize_text(text):
    #text = text.lower()
    #text = re.sub(r"[^\w\s]", "", text)  # Rimuove punteggiatura
    #text = text.strip()  # Rimuove spazi extra
    #return text

In [35]:
# Assicurati che le traduzioni generate siano una lista di stringhe
#hypotheses = sampled_data['translated'].tolist()

# Converti le traduzioni di riferimento in una lista di liste
#references = [[ref] for ref in sampled_data['Target_clean'].tolist()]

# Normalizza hypotheses e references
#hypotheses = [normalize_text(hyp) for hyp in sampled_data['translated'].tolist()]
#references = [[normalize_text(ref)] for ref in sampled_data['Target_clean'].tolist()]

# Calcolo del BLEU score tra la traduzione generata e la traduzione di riferimento
#bleu = sacrebleu.corpus_bleu(hypotheses, references)
#print(f"BLEU score for sample: {bleu.score}")


BLEU score for sample: 100.00000000000004


In [ ]:
# Definisci hypotheses e references per la valutazione
hypotheses = sampled_data['translated'].tolist()
references = [[ref] for ref in sampled_data['Target_clean'].tolist()]

# Calcolo del BLEU con sacrebleu senza smoothing
bleu_no_smoothing = sacrebleu.corpus_bleu(hypotheses, references)
print(f"SacreBLEU score senza smoothing: {bleu_no_smoothing.score}")

# Calcolo del BLEU con sacrebleu e smoothing esponenziale
bleu_exp_smoothing = sacrebleu.corpus_bleu(hypotheses, references, smooth_method="exp")
print(f"SacreBLEU score con smoothing esponenziale: {bleu_exp_smoothing.score}")

# Calcolo del BLEU con sacrebleu su sotto-campioni
sample_bleu_scores = []
num_samples = 5  # Numero di sotto-campioni
sample_size = int(len(hypotheses) * 0.4)  # Usa il 40% del dataset per ogni sotto-campione

for _ in range(num_samples):
    indices = np.random.choice(len(hypotheses), sample_size, replace=False)
    sample_hypotheses = [hypotheses[i] for i in indices]
    sample_references = [[references[i][0]] for i in indices]
    bleu_sample = sacrebleu.corpus_bleu(sample_hypotheses, sample_references, smooth_method="exp")
    sample_bleu_scores.append(bleu_sample.score)

avg_bleu_score = np.mean(sample_bleu_scores)
print(f"SacreBLEU score medio su sotto-campioni: {avg_bleu_score}")

# Calcolo del BLEU con NLTK senza smoothing
nltk_bleu_score = corpus_bleu([[ref[0]] for ref in references], hypotheses)
print(f"NLTK BLEU score senza smoothing: {nltk_bleu_score * 100}")

# Calcolo del BLEU con NLTK e smoothing (metodo 1)
smoothing = SmoothingFunction().method1
nltk_bleu_smoothing = corpus_bleu([[ref[0]] for ref in references], hypotheses, smoothing_function=smoothing)
print(f"NLTK BLEU score con smoothing (method1): {nltk_bleu_smoothing * 100}")

# FINE-TUNING (ANCORA DA VEDERE)

Se le performance non sono soddisfacenti, puoi considerare le seguenti opzioni:

Fine-Tuning del Modello: puoi eseguire un fine-tuning del modello di traduzione con il tuo dataset per migliorare le performance. Questo richiede però un po’ di risorse e tempo su Colab.

Ecco i passaggi base per un fine-tuning:

Carica il dataset in un formato compatibile con il trainer di Hugging Face.
Configura i parametri di addestramento come numero di epoche, batch size, e tasso di apprendimento.
Avvia l'addestramento e monitora le metriche di valutazione.
Aggiustamenti dei Parametri di Decoding: per la generazione delle traduzioni, prova a regolare i parametri come la lunghezza massima della sequenza e la penalità di lunghezza (in modo che il modello eviti traduzioni troppo brevi o prolisse). Puoi anche usare la temperature o il beam search per migliorare la qualità.

In [16]:
# Carica il dataset come un oggetto Dataset di Hugging Face
# Crea un dataset per il fine-tuning
train_data = Dataset.from_pandas(sampled_data[['Source_clean', 'Target_clean']])

# Configura i parametri di training
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True
)

# Funzione di preprocessamento del dataset
def preprocess_function(examples):
    inputs = [f"translate English to Italian: {ex}" for ex in examples["Source_clean"]]
    targets = examples["Target_clean"]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Applica la funzione di preprocessamento
tokenized_train_data = train_data.map(preprocess_function, batched=True)

# Inizializza il Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/7676 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


ValueError: You have set `args.eval_strategy` to epoch but you didn't pass an `eval_dataset` to `Trainer`. Either set `args.eval_strategy` to `no` or pass an `eval_dataset`. 